In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from crazyflie_description_py.parameters import N_BUFFER, FS
from audio_stack.parameters import TUKEY_ALPHA
f = 3000
noise_scale = 0.1 
times = np.arange(0, 1, step=1/FS)
signal_full = np.sin(2*np.pi*f*times) + np.random.normal(scale=noise_scale, size=times.shape)
signal_buffer = signal_full[:N_BUFFER]

In [ ]:
from scipy import signal
freqs = np.fft.rfftfreq(N_BUFFER, 1/FS)

fig, ax =plt.subplots()
fig_f, ax_f =plt.subplots()
fig_w, ax_w =plt.subplots()

buffer_f = np.fft.rfft(signal_buffer)

window_f = np.fft.rfft(np.ones(N_BUFFER))
ax.plot(signal_buffer)
ax_f.plot(freqs, np.abs(buffer_f), label='original')
ax_w.plot(freqs, np.abs(window_f), label='original')

for i, alpha in enumerate(np.logspace(-3, 0, 4)):
    label = f'{alpha:.1e}'
    window_tukey = signal.tukey(N_BUFFER, alpha=alpha)
    signal_buffer_tukey = window_tukey * signal_buffer
    ax.plot(signal_buffer_tukey + 2 * i + 2)
    
    buffer_tukey_f = np.fft.rfft(signal_buffer_tukey)
    ax_f.plot(freqs, np.abs(buffer_tukey_f), label=label)
    
    window_tukey_f = np.fft.rfft(window_tukey)
    ax_w.plot(freqs, np.abs(window_tukey_f), label=label)
    
ax_f.set_xlim(f-1500, f+1500)
ax_w.set_xlim(0, 100)
ax_f.set_yscale('log')
#ax_w.set_yscale('log')
ax_f.legend(loc='upper right')

# Cepstrum

In [ ]:
import scipy.signal
f_gain = 300
f_signal = 3000
noise_scale = 0.1 
times = np.arange(0, 1, step=1/FS)

signal = np.sin(2*np.pi*f_signal*times)
gain = np.sin(2*np.pi*f_gain*times)
signals_full = scipy.signal.convolve(signal, gain, mode='same')
    
if noise_scale > 0:
    signals_full += np.random.normal(scale=noise_scale, size=times.shape)

In [ ]:
plt.plot(signals_full[:N_BUFFER])

In [ ]:
def apply_bandpass(signal_f, bandpass=[1000, 5000], method='hann'):
    window = np.zeros(freqs.shape)
    mask = (freqs > bandpass[0]) & (freqs < bandpass[1])
    if method == 'hann':
        window[mask] = scipy.signal.windows.hann(np.sum(mask))
    elif method == '':
        window[mask] = np.ones(np.sum(mask)) 
    return signal_f * window

cutoff = 200
method = ''
#fft = np.fft.rfft(signals_full)
#freqs = np.fft.rfftfreq(len(signals_full), 1/FS)

fft = signals_full
freqs = times

In [ ]:
plt.figure()
plt.plot(freqs, fft)
plt.title('fft')

#fft_masked = apply_bandpass(np.abs(fft), method=method)
fft_masked = fft

inverse = np.abs(np.fft.irfft(fft_masked, n=len(signals_full)))
plt.figure()
plt.plot(inverse)
plt.title('inverse')

# TODO(FD) why is there a factor of 2 necessary? 
#fft_inverse = np.fft.rfft(valid_inverse)
#freq_inverse = np.fft.rfftfreq(len(valid_inverse), 2/FS)
#plt.figure()
#plt.plot(freq_inverse, np.abs(fft_inverse))
#plt.title('inverse of inverse fft')

In [ ]:
log_fft = np.log10(np.abs(fft)**2)

plt.figure()
plt.plot(freqs, log_fft)
plt.title('log-fft')

#log_fft_masked = apply_bandpass(log_fft, method=method)
log_fft_masked = log_fft
cepstrum = np.abs(np.fft.irfft(log_fft_masked, n=len(signals_full)))

plt.figure()
plt.plot(cepstrum)
plt.title('sepctrum')
